In [1]:
from htm.bindings.sdr import SDR, Metrics
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
import numpy as np
import pandas as pd
import pathlib
import datetime
import csv
from datetime import datetime
import os
from htm.encoders.rdse import RDSE, RDSE_Parameters
import time
import traceback
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn_extra.cluster import KMedoids
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from random import shuffle


In [2]:
inputSources = [
   "monthly_sp500_pca.csv",
   "weekly_dow_jones.csv",
   "weekly_nasdaq.csv",
   "weekly_sp500.csv",
   "monthly_vix_close.csv",
   "monthly_vix_high.csv",
   "monthly_vix_low.csv",
   "monthly_vix_open.csv",
   "daily_natural_gas.csv",
   "daily_oil_prices.csv",
   "value1_vix_close.csv",
   "value1_vix_high.csv",
   "value1_vix_low.csv",
   "value1_vix_open.csv",
   "monthly_gold_prices.csv"
]

In [3]:

config = {
    'enc': {
        "value" :
            {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
        "time": 
            {'timeOfDay': (30, 1), 'weekend': 21}
    },
    'sp': {
        'inputDimensions': None,
        'columnDimensions': (1638,),
        'potentialPct': 0.85,
        'potentialRadius': None,
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False,
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    },
    'anomaly': {'period': 1000},
    'learnRows': 100,
    'reflexSize': 2048,
    'accuracyThreshold': 0.5,
    'controlThreshold': 0.7
}


In [4]:
class ReflexiveMemory:
  def __init__(self, reflexSize, dimensions_dense, dimensions_sparse_sp):
    self.acKey0 = None
    self.pairs = {}
    self.tableSize  = reflexSize
    self.dimensions_dense = dimensions_dense
    self.dimensions_sparse_sp = dimensions_sparse_sp

  def add(self, denseColumns):
    acKey1 = '-'.join(map(str, denseColumns.sparse))
    if(self.acKey0 != None):

      sequence = self.pairs.get(self.acKey0, {})
      sequence_data = sequence.get(acKey1, {
         "count": 0,
         "time": datetime.now()
      })
      sequence_data["count"] = sequence_data["count"] + 1
      sequence_data["time"] = datetime.now()

      if self.pairs.get(self.acKey0, None) is None:
        self.pairs[self.acKey0] = { acKey1: sequence_data }
      else:
        self.pairs[self.acKey0][acKey1] = sequence_data
        
      table_entries = 0
      oldKey1 = None
      oldKey2 = None
      oldTime = datetime.now()
      for key1, value1 in self.pairs.items():
        table_entries = table_entries + len(value1.items())
        for key2, value2 in value1.items():
          if value2['time'] < oldTime:
            oldKey1 = key1
            oldKey2 = key2
            oldTime = value2['time']
      if table_entries > self.tableSize:
        del self.pairs[oldKey1][oldKey2]
        if len(self.pairs[oldKey1].items()) == 0:
          del self.pairs[oldKey1]

    self.acKey0 = acKey1

  def predict(self, denseColumns):
    return_count = 0
    return_sdr = None

    acKey = '-'.join(map(str, denseColumns.sparse))
    sequences = self.pairs.get(acKey, {})
    for sequence_key, sequence_data in sequences.items():
      if sequence_data["count"] > return_count:
        return_count = sequence_data["count"]
        return_sdr = sequence_key

    if return_sdr is not None:
      tmp_sdr = SDR( self.dimensions_dense )
      tmp_sdr.sparse = list(map(int, return_sdr.split('-')))
      return_sdr = tmp_sdr
    else:
      return_count = None

    return return_count, return_sdr


In [5]:
class ControlUnit:
  def __init__(self, controlThreshold):
    self.anomalyRM = []
    self.anomalyTM = []
    self.anomalyNU = []
    self.anomalyCU = []
    self.historyRM = []
    self.historyTM = []
    self.historyGT = []
    self.historyCU = []
    self.controlThreshold = controlThreshold

  def anomalyScore(self, y, x):
      if np.count_nonzero(y) != 0:
          return 1 - np.count_nonzero((x & y)) / np.count_nonzero(y)
      return 1

  def compute(self, denseColumns1, sp, tm, rm):

    if rm.acKey0 is not None:

      denseColumns0 = SDR( rm.dimensions_dense )
      denseColumns0.sparse = list(map(int, rm.acKey0.split('-')))

      tm.activateDendrites(True)
      predictiveCells = tm.getPredictiveCells()

      predictiveColumns = SDR( rm.dimensions_sparse_sp )
      predictiveColumns.sparse = list(set(sorted(list(np.where(predictiveCells.dense == 1)[0]))))

      reflexiveColumns = SDR( rm.dimensions_sparse_sp )
      reflexiveCount, denseReflexiveColumns = rm.predict(denseColumns0)
      if denseReflexiveColumns is not None:
        sp.compute(denseReflexiveColumns, False, reflexiveColumns)

      activeColumns0 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns0, False, activeColumns0)

      activeColumns1 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns1, False, activeColumns1)

      self.historyRM.append( reflexiveColumns.dense )
      self.historyTM.append( predictiveColumns.dense )
      self.historyGT.append( activeColumns1.dense )

      self.anomalyNU.append(tm.anomaly)
      self.anomalyRM.append( self.anomalyScore(activeColumns1.dense, reflexiveColumns.dense) )
      self.anomalyTM.append( self.anomalyScore(activeColumns1.dense, predictiveColumns.dense) )

      #  = sum(self.anomalyRM[-8:])/len(self.anomalyRM[-8:])
      if (len(self.anomalyRM) > 4) and (sum(self.anomalyRM[-5:-1]) > sum(self.anomalyTM[-5:-1])):
        self.anomalyCU.append( self.anomalyTM[-1] )
        self.historyCU.append( self.historyTM[-1] )
      else:
        self.anomalyCU.append( self.anomalyRM[-1] )
        self.historyCU.append( self.historyRM[-1] )

#      if overlapRM < (1 - 0.5):
#        oldKey1 = '-'.join(map(str, denseColumns0.sparse))
#        oldKey2 = '-'.join(map(str, denseReflexiveColumns.sparse))
#        if reflexiveCount > 1:
#          self.pairs[oldKey1][oldKey2]["count"] = reflexiveCount - 1
#        else:
#          del self.pairs[oldKey1][oldKey2]
#          if len(self.pairs[oldKey1].items()) == 0:
#            del self.pairs[oldKey1]
  

In [6]:
input_path = pathlib.Path('../datasets/numenta')
dataset_metrics = []

pbar = tqdm(total=len(inputSources))
for dataset in inputSources:

    tm_infer_tm = 0
    tm_infer_rm = 0

    records = []
    with open(input_path.joinpath(dataset), "r") as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        next(reader)
        next(reader)
        for record in reader:
            records.append(record)
        
    scalarEncoderParams = RDSE_Parameters()
    scalarEncoderParams.size = config["enc"]["value"]["size"]
    scalarEncoderParams.sparsity = config["enc"]["value"]["sparsity"]
    scalarEncoderParams.resolution = config["enc"]["value"]["resolution"]
    scalarEncoder = RDSE( scalarEncoderParams )
    encodingWidth = (scalarEncoder.size)

    config['sp']['inputDimensions'] = (encodingWidth,)
    config['sp']['potentialRadius'] = encodingWidth

    sp = SpatialPooler(
        inputDimensions = config['sp']['inputDimensions'],
        columnDimensions = config['sp']['columnDimensions'],
        potentialPct = config['sp']['potentialPct'],
        potentialRadius = config['sp']['potentialRadius'],
        globalInhibition = config['sp']['globalInhibition'],
        localAreaDensity = config['sp']['localAreaDensity'],
        synPermInactiveDec = config['sp']['synPermInactiveDec'],
        synPermActiveInc = config['sp']['synPermActiveInc'],
        synPermConnected = config['sp']['synPermConnected'],
        boostStrength = config['sp']['boostStrength'],
        wrapAround = config['sp']['wrapAround'],
        seed = config['sp']['seed']
    )

    tm = TemporalMemory(
        columnDimensions = config['sp']['columnDimensions'],
        cellsPerColumn = config['tm']['cellsPerColumn'],
        activationThreshold = config['tm']['activationThreshold'],
        initialPermanence = config['tm']['initialPermanence'],
        connectedPermanence = config['sp']['synPermConnected'],
        minThreshold = config['tm']['minThreshold'],
        maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
        permanenceIncrement = config['tm']['permanenceIncrement'],
        permanenceDecrement = config['tm']['permanenceDecrement'],
        predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
        maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
        maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
    )

    rm = ReflexiveMemory( config['reflexSize'] , config['sp']['inputDimensions'], config['sp']['columnDimensions'])
    cu = ControlUnit( config['controlThreshold'] )

    try:
        
        for count, record in enumerate(records):
            
            learn_sp = config['sp']['learn']
            learn_tm = config['tm']['learn']
            if count < config['learnRows']:
                learn_sp = True
                learn_tm = True

            consumption = float(record[1])
            consumptionBits = scalarEncoder.encode(consumption)
            encoding = SDR( consumptionBits )

            cu.compute(encoding, sp, tm, rm)

            rm.add(encoding)

            activeColumns = SDR( sp.getColumnDimensions() )
            sp.compute(encoding, learn_sp, activeColumns)

            tmp_tm = time.time()
            tm.compute(activeColumns, learn=learn_tm)
            tm_infer_tm = tm_infer_tm + (time.time() - tmp_tm)

            tmp_tm = time.time()
            rm.predict(encoding)
            tm_infer_rm = tm_infer_rm + (time.time() - tmp_tm)

    except Exception as e:
        print(traceback.format_exc())
        print(e)

    def roc_auc_score_multiclass(y_true, y_pred):
        scores = []
        for y_class in set(y_true):
            y_true_class = [True if x == y_class else False for x in y_true]
            y_pred_class = [True if x == y_class else False for x in y_pred]
            scores.append(roc_auc_score(y_true_class, y_pred_class))
        return sum(scores) / len(scores)

    def match(y, x, idx1, cu, accuracyThreshold):
        n_samples = len(y)
        score1 = cu.anomalyScore(y[idx1], x[idx1])
        if score1 > (1 - accuracyThreshold):
            idx_closest = None
            score_closest = None
            for idx2 in range(n_samples):
                score2 = cu.anomalyScore(y[idx2], x[idx1])
                if score_closest is None or score_closest > score2:
                    score_closest = score2
                    idx_closest = idx2
            return idx_closest
        return idx1

    def calculateMetricsAnomaly(anomaly_scores, config, suffix):

        metric = {}

        anomaly_probability = []
        anomaly_history = AnomalyLikelihood(config["anomaly"]["period"])
        for anomaly_value in anomaly_scores:
            anomaly_probability.append( anomaly_history.compute(anomaly_value) )

        metric['anomaly-avg-'+suffix] = sum(anomaly_scores) / len(anomaly_scores)
        metric['anomaly-samples-'+suffix] = len(anomaly_scores)
        metric['anomaly-prob-avg-'+suffix] = np.count_nonzero(anomaly_probability) / len(anomaly_probability)

        return metric

    def calculateMetrics(Y_dataset, X_dataset, anomaly_scores, total_infe_time, config, cu, suffix):

        metric = {}

        n_samples = len(Y_dataset)

        Y_labels = list(range(n_samples))
        X_labels = [ match(Y_dataset, X_dataset, idx, cu, config['accuracyThreshold']) for idx in range(n_samples)]
        precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
        metric['total-infer-time-'+suffix] = total_infe_time
        metric['infer-time-'+suffix] = total_infe_time / len(anomaly_scores)
        metric['accuracy-'+suffix] = accuracy_score(Y_labels, X_labels)
        metric['precision-'+suffix] = precision
        metric['recall-'+suffix] = recall
        metric['fscore-'+suffix] = fscore
        metric['support-'+suffix] = support
        metric['auc-'+suffix] = roc_auc_score_multiclass(Y_labels, X_labels)

        metric.update( calculateMetricsAnomaly(anomaly_scores, config, suffix) )

        return metric
    
    tm_infer_cu = 0
    avg_infer_time_rm = tm_infer_rm / len(cu.anomalyRM)
    avg_infer_time_tm = tm_infer_tm / len(cu.anomalyTM)
    for idx, eval_anomaly in enumerate(cu.anomalyCU):
        if eval_anomaly == cu.anomalyRM[idx]:
            tm_infer_cu = tm_infer_cu + avg_infer_time_rm
        else:
            tm_infer_cu = tm_infer_cu + avg_infer_time_tm

    metric = {}
    metric['dataset'] = dataset
    metric.update( calculateMetrics(cu.historyGT, cu.historyRM, cu.anomalyRM, tm_infer_rm, config, cu, 'rm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyTM, cu.anomalyTM, tm_infer_tm, config, cu, 'tm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyCU, cu.anomalyCU, tm_infer_cu, config, cu, 'cu') )
    metric.update( calculateMetricsAnomaly(cu.anomalyNU, config, 'nupic') )

    dataset_metrics.append(metric)
    pbar.update(1)
    # break

pbar.close()

100%|██████████| 15/15 [05:41<00:00, 22.77s/it]


In [7]:
df = pd.DataFrame(dataset_metrics)
df.to_csv('metrics.csv', index=False)

In [8]:
table_1_features = ['dataset','accuracy-cu','accuracy-rm','accuracy-tm','anomaly-avg-cu','anomaly-avg-rm','anomaly-avg-tm','anomaly-avg-nupic']
df[table_1_features]

,dataset,accuracy-cu,accuracy-rm,accuracy-tm,anomaly-avg-cu,anomaly-avg-rm,anomaly-avg-tm,anomaly-avg-nupic
0,monthly_sp500_pca.csv,0.402803,0.174893,0.407069,0.585593,0.804159,0.580134,0.580134
1,weekly_dow_jones.csv,0.035594,0.011063,0.036075,0.939194,0.984775,0.936100,0.936100
2,weekly_nasdaq.csv,0.213462,0.070673,0.215385,0.763401,0.907959,0.760503,0.760557
3,weekly_sp500.csv,0.342624,0.135512,0.345988,0.625467,0.843038,0.623345,0.623398
4,monthly_vix_close.csv,0.969136,0.755802,0.973333,0.047912,0.315922,0.041516,0.041763
5,monthly_vix_high.csv,0.951358,0.720494,0.955556,0.073330,0.347647,0.065830,0.066077
6,monthly_vix_low.csv,0.976296,0.766667,0.981235,0.041656,0.292939,0.033553,0.033800
7,monthly_vix_open.csv,0.950617,0.689383,0.957037,0.070782,0.344928,0.062455,0.062647
8,daily_natural_gas.csv,0.982580,0.955157,0.996551,0.024264,0.063362,0.008686,0.008858
9,daily_oil_prices.csv,0.990964,0.913855,0.994337,0.030080,0.215378,0.020167,0.020288


In [9]:
table_2_features = ['dataset','anomaly-prob-avg-cu','anomaly-prob-avg-rm','anomaly-prob-avg-tm','anomaly-prob-avg-nupic','anomaly-samples-cu','anomaly-samples-rm','anomaly-samples-tm','anomaly-samples-nupic']
df[table_2_features]

,dataset,anomaly-prob-avg-cu,anomaly-prob-avg-rm,anomaly-prob-avg-tm,anomaly-prob-avg-nupic,anomaly-samples-cu,anomaly-samples-rm,anomaly-samples-tm,anomaly-samples-nupic
0,monthly_sp500_pca.csv,0.391225,0.391225,0.391225,0.391225,1641,1641,1641,1641
1,weekly_dow_jones.csv,0.519481,0.519481,0.519481,0.519481,2079,2079,2079,2079
2,weekly_nasdaq.csv,0.519712,0.519712,0.519712,0.519712,2080,2080,2080,2080
3,weekly_sp500.csv,0.519942,0.519942,0.519942,0.519942,2081,2081,2081,2081
4,monthly_vix_close.csv,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
5,monthly_vix_high.csv,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
6,monthly_vix_low.csv,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
7,monthly_vix_open.csv,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
8,daily_natural_gas.csv,0.827699,0.827699,0.827699,0.827699,5798,5798,5798,5798
9,daily_oil_prices.csv,0.879639,0.879639,0.879639,0.879639,8300,8300,8300,8300


In [10]:
table_3_features = ['dataset','infer-time-cu','infer-time-rm','infer-time-tm','total-infer-time-cu','total-infer-time-tm','total-infer-time-rm']
df[table_3_features]

,dataset,infer-time-cu,infer-time-rm,infer-time-tm,total-infer-time-cu,total-infer-time-tm,total-infer-time-rm
0,monthly_sp500_pca.csv,0.000251,0.000010,0.000331,0.412509,0.542461,0.017211
1,weekly_dow_jones.csv,0.000109,0.000008,0.000302,0.226311,0.626922,0.017653
2,weekly_nasdaq.csv,0.000224,0.000009,0.000347,0.466896,0.720849,0.019360
3,weekly_sp500.csv,0.000251,0.000010,0.000316,0.523150,0.657724,0.019801
4,monthly_vix_close.csv,0.000147,0.000010,0.000226,0.596016,0.915727,0.040250
5,monthly_vix_high.csv,0.000227,0.000011,0.000338,0.919030,1.370492,0.044590
6,monthly_vix_low.csv,0.000103,0.000009,0.000170,0.415578,0.688610,0.036618
7,monthly_vix_open.csv,0.000198,0.000011,0.000300,0.803869,1.214373,0.042745
8,daily_natural_gas.csv,0.000011,0.000007,0.000049,0.065895,0.286250,0.042244
9,daily_oil_prices.csv,0.000060,0.000008,0.000106,0.497993,0.877300,0.070471


In [11]:
table_4_features = ['dataset'] + sorted(list(set(df.columns) - (set(table_1_features) | set(table_2_features) | set(table_3_features))))
df[table_4_features]

,dataset,auc-cu,auc-rm,auc-tm,fscore-cu,fscore-rm,fscore-tm,precision-cu,precision-rm,precision-tm,recall-cu,recall-rm,recall-tm,support-cu,support-rm,support-tm
0,monthly_sp500_pca.csv,0.701220,0.587195,0.703354,0.386305,0.168687,0.389605,0.379974,0.167070,0.382960,0.402803,0.174893,0.407069,None,None,None
1,weekly_dow_jones.csv,0.517565,0.505294,0.517806,0.032228,0.009941,0.032549,0.030945,0.009700,0.031265,0.035594,0.011063,0.036075,None,None,None
2,weekly_nasdaq.csv,0.606542,0.535113,0.607504,0.203521,0.067207,0.204995,0.199640,0.066046,0.200963,0.213462,0.070673,0.215385,None,None,None
3,weekly_sp500.csv,0.671154,0.567548,0.672837,0.327580,0.131012,0.331360,0.321562,0.129345,0.325487,0.342624,0.135512,0.345988,None,None,None
4,monthly_vix_close.csv,0.984564,0.877871,0.986663,0.965417,0.751908,0.970360,0.964033,0.751306,0.969152,0.969136,0.755802,0.973333,None,None,None
5,monthly_vix_high.csv,0.975673,0.860212,0.977772,0.947397,0.717005,0.952516,0.946208,0.716555,0.951390,0.951358,0.720494,0.955556,None,None,None
6,monthly_vix_low.csv,0.988145,0.883305,0.990615,0.973750,0.763504,0.978979,0.972690,0.762800,0.978019,0.976296,0.766667,0.981235,None,None,None
7,monthly_vix_open.csv,0.975303,0.844653,0.978513,0.947201,0.685625,0.954395,0.946136,0.684961,0.953431,0.950617,0.689383,0.957037,None,None,None
8,daily_natural_gas.csv,0.991289,0.977575,0.998275,0.982001,0.954279,0.995853,0.981908,0.954123,0.995580,0.982580,0.955157,0.996551,None,None,None
9,daily_oil_prices.csv,0.995481,0.956923,0.997168,0.989778,0.910664,0.993237,0.989279,0.909916,0.992724,0.990964,0.913855,0.994337,None,None,None


In [ ]:
df['cu-infer-speedup'] = 1 - (df['total-infer-time-cu'] / df['total-infer-time-tm'])
df['cu-accuracy-improvement'] = df['accuracy-cu'] - df['accuracy-tm']
table_5_features = ['dataset','cu-accuracy-improvement','cu-infer-speedup','accuracy-cu','accuracy-rm','accuracy-tm','total-infer-time-cu','total-infer-time-tm','total-infer-time-rm']
df[table_5_features]

,dataset,cu-accuracy-improvement,cu-infer-speedup,accuracy-cu,accuracy-rm,accuracy-tm,total-infer-time-cu,total-infer-time-tm,total-infer-time-rm
0,monthly_sp500_pca.csv,-0.004266,0.239561,0.402803,0.174893,0.407069,0.412509,0.542461,0.017211
1,weekly_dow_jones.csv,-0.000481,0.639013,0.035594,0.011063,0.036075,0.226311,0.626922,0.017653
2,weekly_nasdaq.csv,-0.001923,0.352297,0.213462,0.070673,0.215385,0.466896,0.720849,0.019360
3,weekly_sp500.csv,-0.003364,0.204605,0.342624,0.135512,0.345988,0.523150,0.657724,0.019801
4,monthly_vix_close.csv,-0.004198,0.349134,0.969136,0.755802,0.973333,0.596016,0.915727,0.040250
5,monthly_vix_high.csv,-0.004198,0.329416,0.951358,0.720494,0.955556,0.919030,1.370492,0.044590
6,monthly_vix_low.csv,-0.004938,0.396497,0.976296,0.766667,0.981235,0.415578,0.688610,0.036618
7,monthly_vix_open.csv,-0.006420,0.338038,0.950617,0.689383,0.957037,0.803869,1.214373,0.042745
8,daily_natural_gas.csv,-0.013970,0.769797,0.982580,0.955157,0.996551,0.065895,0.286250,0.042244
9,daily_oil_prices.csv,-0.003373,0.432357,0.990964,0.913855,0.994337,0.497993,0.877300,0.070471
